# Tutorial de 5 minutos

La manera mas accesible de usar `pydap` es como `cliente` de acceso a datos cientificos en servidores de [OPeNDAP](https://www.opendap.org/). Para ello, puedes utilizar `pydap` de directamente por medio del metodo  `open_url`, or usa `pydap` como [engine](https://docs.xarray.dev/en/stable/user-guide/io.html#opendap) por medio de `xarray` directamente. `xarray` hace possible el uso de las herramientas del ecosistema de [Pangeo](https://pangeo.io/).

## OPeNDAP - La vision original
La vision original de [OPeNDAP](https://www.opendap.org/) ([Cornillion, et al 1993](https://zenodo.org/records/10610992)) fue el hacer la equivalencia:

$ \;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\; \boxed{\text{URL} \approx \text{Dataset Remoto} }$


Y ademas, 

$ \;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\; \boxed{\text{URL + Restricciones} \approx \text{Subregion de un Dataset Remoto}} $


En este corto tutorial demostraremos el accesso a informacion cientifica en un servidor de OPeNDAP por medio de

- `pydap`
- `xarray`

For more information about [OPeNDAP](https://www.opendap.org/) and Hyrax you can go to the official [OPeNDAP documentation](https://opendap.github.io/documentation/UserGuideComprehensive.html).

El dataset remote que utilizaremos en este tutorial puede ser inspeccionado [aqui](http://test.opendap.org:8080/opendap/tutorials/20220531090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc.dmr.html)


In [ ]:
from pydap.client import open_url
import xarray as xr
import numpy as np

Ahora definimos el `URL` que apunta al dataset remoto en el servidor de `OPeNDAP`.

In [ ]:
url = "http://test.opendap.org:8080/opendap/tutorials/20220531090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc"

## `PyDAP`
Empezamos primero el acceso al archivo usando solamente `Pydap`

In [ ]:
pydap_ds = open_url(url, protocol='dap4')

```{note}
Ademas del argumento `url` que define al dataset, tambien definimos el argumento `protocol="dap4"`. Esto hace referencia al Protocol de OPeNDAP. Otra opcion es:  `protocol='dap2'`. 
```

```{note}
Existen muchos servidores de OPeNDAP,pero solo 2 de ellos implementan el protocolo `DAP4`. Cualquier servidor que implemente `DAP4` tambien implementa `DAP2`. En este tutorial nos enfocaremos en `DAP4`. 
```


`pydap` descarga del servidor OPeNDAP los metadatos, es decir, la informacion que describe los contenidos del archivo remote. Sin embargo, ningun dato numerico or binario a sido descargado hasta este momento. Para visualizar las variables que existen dentro del archivo, executamos el metodo `.tree()`:




In [ ]:
pydap_ds.tree()

In [ ]:
pydap_ds['sst_anomaly'].shape

In [ ]:
print('La variable numerica `sst_anomaly` ocupa: ', pydap_ds['sst_anomaly'].nbytes/1e9, '[GBs] de memoria')

```{note}
Solo los metadatos han sido descargados. `PyDAP` processa estos metadatos del URL remoto para create el `Dataset` que hace referencia al contenido del archivo remoto. 
```

Cada variable contiene `atributos` que describen los valores, y algunas de las transformaciones que deben efectuarse para darle sentido fisico a las variables en si. Por ejemplo,  `scale_factor`, `offsets` and `_FillValue`.


In [ ]:
pydap_ds['sst_anomaly'].attributes

Los metadatos cientificos siguen las convenciones definidas en: [NetCDF Climate and Forcasts (CF) Metadata Conventions](https://cfconventions.org/cf-conventions/cf-conventions.html).


### **Como Descargar el Arreglo numerico de la variable remota**

Como se menciono, `Pydap` no ha descargado ningun arreglo numerico. Para descargar los valores de digamos una vbariable de interes, uno debe de indexar la varible de `Pydap`. Por ejemplo, la variable

```python
 pydap_ds['sst_anomaly']
```
tiene las siguientes dimensiones: `(1, 17999, 36000)`. Uno puede descarga una fraccion de la variable, por ejemplo el elemento `0` de la primera dimension, los primeros `10` elementos de la segunda dimension, y los primeros `10` de la tercera dimension, de la manera siguiente:

In [ ]:
%%time
array = pydap_ds['sst_anomaly'][0, 0:10, 0:10]

In [ ]:
np.shape(array)

Como se demonstro, el nuevo tamano de la variable `array` es `(1, 10, 10)`. 

Al descargar la variable `sst_anomaly` y assignarla al objecto: `array`, este objeto no es todavia un arreglo NumPy, sino es un `BaseType` del model de `pydap`:

In [ ]:
type(array)

Para extraer el arreglo NumPy de cada `BaseType`, uno tiene que executar la siguiente instruccion:

In [ ]:
data = array.data

In [ ]:
type(data)

## Utiliza el Servidor Remoto

Cuando el usuario hace la siguiente operacion:

```python
pydap_ds['sst_anomaly'][0, 0:10, 0:10]
``

Lo que `Pydap` hace internamente es generar el URL con la Expression de Restriccion. EL Servidor de OPeNDAP reconoce este URL y manda la informacion numerica especificada en el resultante URL. Eso significa que el archivo completo nunca fue descargado! Solo parte espeficidada en el URL. En este caso, Pydap incluyo la siguiente informacion al URL:

```python
<OPeNDAP_URL> + "?dap4.ce=\sst_anomaly[0][0:1:9][0:1:9]"
```
Un usuario puede realizar esta operacion manual, pero `pydap` facilita y automatiza este procedimiento. El URL de arriba implica que de todo el dataset original, solo la variable `sst_anomaly` es requerida, y de todo el dominio de la variable, solo la region especificada por las indexes `[0][0:1:9][0:1:9]` debe ser mandada por el servidor OPeNDAP. ENtonces, es el Servidor OPeNDAP remote el que realiza la operacion de abrir y mandar la informacion numerica al usuario. Lo indexes en el URL implican

- El primer elemento dela primera dimension (en este caso, `time`).
- `[0:1:9]` indica los primeros `10` elementos de la segunda dimension (llamada `lat`).
- `[0:1:9]` indica los primeros `10` elementos de la tercera dimension (llamada `lon`).

### 

A continuacion, utilizamos `pydap` con el URL que incluye la condicion de Restriccion (CE) para descargar las variables `lat` y `sst_anomaly`




In [ ]:
CE = "?dap4.ce=/lat;/sst_anomaly[0][0:1:9][0:1:9]"

In [ ]:
pydap_ds = open_url(url+CE, protocol='dap4')

In [ ]:
pydap_ds.tree()

In [ ]:
pydap_ds['sst_anomaly'].shape

In [ ]:
pydap_ds['lat'].shape

```{note}
El servidor de `OPeNDAP` solo applico la restriccion espacial a la variable `sst_anomaly`, mientras que `lat` (y cualquier otra) mantuvo su tamano original. Para asegurar que tambien la variable `lat` mantenga el mismo tamano en la dimension compartida con `sst_anomaly`, uno puede especicar en el URL que `lat` tambien debe ser restringida espacialmente.
```

# Xarray 

`PyDAP`se puede utilizar internamente desde `xarray`, al definir el parametro

```python
engine='pydap'
```

```{tip}
Para especificar que protocolo `dap2` o `dap4` se debe usar en `xarray` cuando se especifica `engine=pydap`, hay que reemplazar el esquema del URL `https` con `dap2` o `dap4`. 
```


In [ ]:
'dap4'+url[4:]

In [ ]:
dataset = xr.open_dataset('dap4'+url[4:], engine='pydap')
dataset